In [1]:
import configparser
import psycopg2
import boto3
import pandas as pd
import json

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

host = config.get('CLUSTER','HOST')
dbname = config.get("CLUSTER", 'DB_NAME')
user = config.get('CLUSTER', 'DB_USER')
password = config.get('CLUSTER', 'DB_PASSWORD')
port = config.get('CLUSTER', 'DB_PORT')

arn = config.get('IAM_ROLE', 'ARN')
DWH_CLUSTER_IDENTIFIER = 'MydwhCluster'

PLEASE SPECIFY YOUR AWS CREDENTIALS TO PROCEED

In [3]:
KEY = ''
SECRET = ''

Check IAMROLE

In [4]:
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

In [5]:
roleArn = iam.get_role(RoleName='myRedshiftRole')['Role']['Arn']

In [6]:
roleArn

'arn:aws:iam::570606663173:role/myRedshiftRole'

Launch Redshift

In [8]:
redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [8]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType='multi-node',
        NodeType='dc2.large',
        NumberOfNodes=4,

        #Identifiers & Credentials
        DBName='mydwh',
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername='redshift_user',
        MasterUserPassword='Passw0rd',
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

Check cluster status - until status become 'available'

In [9]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,mydwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,redshift_user
4,DBName,mydwh
5,Endpoint,"{'Address': 'mydwhcluster.c0wwgdvjxudl.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-4d12a535
7,NumberOfNodes,4


In [10]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  mydwhcluster.c0wwgdvjxudl.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::570606663173:role/myRedshiftRole


Try and read S3

In [11]:
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

In [12]:
project_s3 = s3.Bucket('udacity-dend')

In [33]:
for obj in project_s3.objects.filter(Prefix="log_json_path.json"):
    json_path = obj
file_content = json_path.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

In [84]:
json_content

{'jsonpaths': ["$['artist']",
  "$['auth']",
  "$['firstName']",
  "$['gender']",
  "$['itemInSession']",
  "$['lastName']",
  "$['length']",
  "$['level']",
  "$['location']",
  "$['method']",
  "$['page']",
  "$['registration']",
  "$['sessionId']",
  "$['song']",
  "$['status']",
  "$['ts']",
  "$['userAgent']",
  "$['userId']"]}

test connection

In [15]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

delete redshift

In [16]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'mydwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'redshift_user',
  'DBName': 'mydwh',
  'Endpoint': {'Address': 'mydwhcluster.c0wwgdvjxudl.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2019, 7, 2, 17, 23, 28, 7000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-d2922d9e',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-4d12a535',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'thu:07:30-thu:08:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': Fal

In [19]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,mydwhcluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,redshift_user
4,DBName,mydwh
5,Endpoint,{'Port': 5439}
6,VpcId,vpc-4d12a535
7,NumberOfNodes,4
